#**Space-based Tokenization**

In [ ]:
import nltk

with open ('shakespeare.txt', 'r') as f:
  text = f.read()


pattern = r'''(?x)          # set flag to allow verbose regexps
        (?:[A-Z]\.)+        # abbreviations, e.g. U.S.A.
        | \w+'\w+           # contractions
      | \w+(?:-\w+)*        # words with optional internal hyphens
      | \$?\d+(?:\.\d+)?%?  # currency and percentages, e.g. $12.40, 82%
      | \.\.\.              # ellipsis
      | [][.,;"'?():_`-]    # these are separate tokens; includes ], [
    '''
vocab = nltk.regexp_tokenize(text, pattern)
vocab = [word.lower() for word in vocab]
print(vocab)
unique_words = set(vocab)
print(unique_words)

FileNotFoundError: [Errno 2] No such file or directory: 'shakespeare.txt'

In [ ]:
from collections import OrderedDict
from collections import Counter

c = Counter(vocab)

sorted_dict = {key: value for key, value in sorted(
    c.items(), key=lambda item: item[1], reverse=True)}
print( sorted_dict.items() )

In [ ]:
sorted_dict

# Byte Pair Encoding

- make Shakespeare into test 1% and train
- train the segmenter with varying k (try normalization strategies)
- compare the performance against a different set
- figure out a measure for accuracy

In [ ]:
import numpy as np
from collections import defaultdict

from collections import OrderedDict
from collections import Counter


def bpe(dictionary, k):
  # just a test!
  # initialise empty dictionary to I count adjacent tokens, not individual ones
  #word_freqs = defaultdict(int)

  # get all unique characters in original corpus
  all_keys = "_ ".join(dictionary.keys())
  vocab_bpe = list(set(all_keys))

  # Corpus/dictionary in einzelne tokens splitten, nach jedem Wort (VOR space!) "_" einfügen
    # worte als list of characters
  dict_matrix = []
  for key, value in dictionary.items():
    new_key = list(f"{str(key)}_ ")
    dict_matrix.append([new_key, value])


  # NICHT corpus, sondern liste an Wörtern in einzelne tokens splitten,
  # >> jede occurence mit counts der Worte multiplizieren
  token_freq = defaultdict(int)

  for iteration in range(k):
    for token_list, value in dict_matrix:
      for i in range(len(token_list)-2):
        # wollen den und den nächsten token als key
        search_key = token_list[i] + token_list[i+1]
        # zu dictionary hinzufügen falls key noch nicht existiert
        token_freq[search_key] += value
    # word_freqs: gehen jede existierende Folge aus zwei tokens in list of words von vorne bis hinten durch

    c = Counter(token_freq)

    sorted_token_freq = {key: value for key, value in sorted(
        c.items(), key=lambda item: item[1], reverse=True)}

    # Find the most frequent token not already in vocab_bpe
    for token in sorted_token_freq.keys():
      if token not in vocab_bpe:
        first_token = token
        break
      else:
        first_token = None  # Optional: fallback in case all tokens are already in vocab

    if first_token:
      vocab_bpe.append(first_token)
      print("New token added:", first_token)
    else:
      print("No new token to add.")

    # höchster count wird gemerged:
    # add to vocab
    #first_token = list(sorted_token_freq.keys())[0]
    #vocab_bpe.append(first_token)
    # replace in list of words
    # start again?
    for i in range(len(dict_matrix)):
        token_list, value = dict_matrix[i]
        j = 0
        while j < len(token_list) - 1:
            search_key = token_list[j] + token_list[j + 1]
            if search_key == first_token:
                merged_token = search_key
                # Merge the tokens
                token_list = token_list[:j] + [merged_token] + token_list[j + 2:]
                # Don't increment j — might be able to merge again
            else:
                j += 1
        dict_matrix[i][0] = token_list



  return vocab_bpe

In [ ]:
bpe(sorted_dict, 9)

New token added: zd
New token added: gy
New token added: d_
New token added: zdg
New token added: zdgy
New token added: xy
New token added: g_
New token added: zdgyd_
New token added: zdg_


['y',
 'z',
 '_',
 'g',
 'x',
 ' ',
 'd',
 'zd',
 'gy',
 'd_',
 'zdg',
 'zdgy',
 'xy',
 'g_',
 'zdgyd_',
 'zdg_']

In [ ]:
import numpy as np

from collections import defaultdict

from collections import OrderedDict
from collections import Counter



sorted_dict = {"xyzdg": 2, "zdgyd": 4}
all_keys = "".join(sorted_dict.keys())
print(all_keys)
vocab_bpe = list(set(all_keys))
print(vocab_bpe)

#make matrix out of dictionary
dict_matrix = []
for key, value in sorted_dict.items():
  new_key = list(f"{str(key)}_ ")
  dict_matrix.append([new_key, value])

print(dict_matrix)

token_freq = defaultdict(int)
for token_list, value in dict_matrix:
  for i in range(len(token_list)-2):
    # wollen den und den nächsten token als key
    search_key = token_list[i] + token_list[i+1]
    # zu dictionary hinzufügen falls key noch nicht existiert
    token_freq[search_key] += value


print(token_freq.items())

c = Counter(token_freq)

sorted_token_freq = {key: value for key, value in sorted(
    c.items(), key=lambda item: item[1], reverse=True)}

print(sorted_token_freq.items())
print(list(sorted_token_freq.keys())[0])

first_token = list(sorted_token_freq.keys())[0]

vocab_bpe.append(first_token)
print(vocab_bpe)

for i in range(len(dict_matrix)):
    token_list, value = dict_matrix[i]
    j = 0
    while j < len(token_list) - 1:
        search_key = token_list[j] + token_list[j + 1]
        if search_key == first_token:
            merged_token = search_key
            # Merge the tokens
            token_list = token_list[:j] + [merged_token] + token_list[j + 2:]
            # Don't increment j — might be able to merge again
        else:
            j += 1
    dict_matrix[i][0] = token_list

"""for token_list, value in dict_matrix:
  for i in range(len(token_list)-2):
    # wollen den und den nächsten token als key
    search_key = token_list[i] + token_list[i+1]
    if search_key == first_token:
      new_token = ["".join([token_list[i], token_list[i+1]])]
      token_list = token_list[:i] + new_token + token_list[i+1:]
      print(new_token)"""



print(f"Dict matrix: {dict_matrix}")

xyzdgzdgyd
['y', 'z', 'g', 'x', 'd']
[[['x', 'y', 'z', 'd', 'g', '_', ' '], 2], [['z', 'd', 'g', 'y', 'd', '_', ' '], 4]]
dict_items([('xy', 2), ('yz', 2), ('zd', 6), ('dg', 6), ('g_', 2), ('gy', 4), ('yd', 4), ('d_', 4)])
dict_items([('zd', 6), ('dg', 6), ('gy', 4), ('yd', 4), ('d_', 4), ('xy', 2), ('yz', 2), ('g_', 2)])
zd
['y', 'z', 'g', 'x', 'd', 'zd']
Dict matrix: [[['x', 'y', 'zd', 'g', '_', ' '], 2], [['zd', 'g', 'y', 'd', '_', ' '], 4]]
